In [1]:
from bs4 import BeautifulSoup
import requests
import re
from os import path
import os

import photoshop.api as ps
from photoshop import Session
import win32com.client

import time

In [2]:
def download_img_website(link):
    response = requests.get(link)
    file = open(path.basename(link), "wb")
    file.write(response.content)
    file.close()

In [3]:
def netonnet_website(id):
    website = requests.get('http://netonnet.se/i/'+str(id))    
    soup = BeautifulSoup(website.content, 'html.parser', from_encoding='utf-8')
    
    rubrik = soup.h2.text
    
    pris = soup.find(class_='price-big').text
    pris = pris.strip()
    
    artikel = soup.find('div', class_='subTitle big').h1.text
    
    uspar = soup.find('div', class_='product-usps').ul.text    
    usparclean = uspar.strip()
    usparclean = usparclean.split('\n')    
    
    
    #print ("rubrik: "+rubrik)
    #print ("pris: "+pris)
    #print ("artikel: "+artikel)
    #print ("Uspar: %s" % usparclean)
    
    bildlink = soup.find(class_='productImage')
    
    bild = r"http://netonnet.se/%s" % bildlink.get("src")
    bildnamn = path.basename(bild)
    
    download_img_website(bild)
        
    #print ("Bild downloaded: %s" % bildnamn)

    return [rubrik, pris, artikel, usparclean, bildnamn]

In [4]:
def import_picture_to_ps(file_path):
    with Session() as pss:
        desc = pss.ActionDescriptor
        desc.putPath(pss.app.charIDToTypeID("null"), file_path)
        event_id = pss.app.charIDToTypeID("Plc ")  # `Plc` need one space in here.        
        pss.app.executeAction(pss.app.charIDToTypeID("Plc "), desc)

In [5]:
def import_text_to_ps(text_list):    
    if type(text_list) is list:
        for item in text_list[0:2]:
            with Session() as pss:
                doc = pss.active_document
                text_color = pss.SolidColor()
                text_color.rgb.red = 0
                text_color.rgb.green = 0
                text_color.rgb.blue = 0
                new_text_layer = doc.artLayers.add()
                new_text_layer.kind = pss.LayerKind.TextLayer
                new_text_layer.textItem.contents = item
                new_text_layer.textItem.position = [160, 167]
                new_text_layer.textItem.size = 40
                new_text_layer.textItem.color = text_color
                assert new_text_layer.textItem.color.rgb.red == text_color.rgb.red
    else:
        with Session() as pss:
            doc = pss.active_document
            text_color = pss.SolidColor()
            text_color.rgb.red = 0
            text_color.rgb.green = 0
            text_color.rgb.blue = 0
            new_text_layer = doc.artLayers.add()
            new_text_layer.kind = pss.LayerKind.TextLayer
            new_text_layer.textItem.contents = text_list
            new_text_layer.textItem.position = [160, 167]
            new_text_layer.textItem.size = 40
            new_text_layer.textItem.color = text_color
            assert new_text_layer.textItem.color.rgb.red == text_color.rgb.red

In [6]:
def import_to_ps(item_id):
    all_the_list = netonnet_website(item_id)
    ROOT_DIR = path.abspath(os.curdir)
    bild_file =  ROOT_DIR+r"\%s" % all_the_list[-1]
    app = ps.Application()
    doc = app.documents.add(977, 415)
    
    import_picture_to_ps(bild_file) #fel bild just nu!
    
    for i in all_the_list[0:4]:
        import_text_to_ps(i)

In [7]:
#netonnet_website(1015891)

In [8]:
#import_to_ps(1013353)

In [9]:
#import_to_ps(1015889)

In [11]:
#import_to_ps(1003370)